In [37]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import tensorflow as tf

from skimage import color, exposure, filters, io, morphology, util
from math import sqrt
from numpy import loadtxt
from tensorflow import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

### A. _Fully Connected Neural Networks_
1. Lakukan klasifikasi untuk dataset cifar menggunakan **_Fully Connected Neural Networks_** dengan arsitektur dasar sebagai berikut:
    - _Fully Connected Layer_ ($1000$ _hidden node_).
    - _Activation Layer_ ($\text{ReLU function}$).
    - _Fully Connected Layer_ ($1000$ _hidden node_)
    - _Activation Layer_ ($\text{ReLU function}$).
    - _Output Layer_ (10 kelas).
    - _Activation Layer_ ($\text{Softmax function}$).
    - _Classification Result_.

Fitur yang digunakan bebas (_handcrafted_/_non-convolution_). Hitung metrik ($\text{accuracy}$, $\text{sensitivity}$, $\text{specificity}$, dan $\text{f1-score}$) untuk data train dan data test.

#### Mendefinisikan Fungsi Bantuan
Untuk memudahkan implementasi, berikut beberapa fungsi bantuan yang akan dipakai

##### Menghitung $\text{recall}$ atau $\text{sensitivity}$
$$\text{recall} = \frac{t_p}{t_p+t_n}$$

In [63]:
def recall(y_true, y_pred): 
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  return true_positives / (possible_positives + K.epsilon())

##### Menghitung $\text{precision}$
$$\text{precision} = \frac{t_p}{t_p + f_p}$$

In [64]:
def precision(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  return true_positives / (predicted_positives + K.epsilon())

##### Menghitung $\text{specificity}$
$$\text{specificity} = \frac{t_n}{t_n + f_p}$$

In [65]:
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

##### Menghitung $f_{1}\text{ Score}$
Kita tau bahwa
$$f_{\beta} = (1 + \beta^{2}) \cdot \frac{\text{precition} \cdot \text{recall}}{(\beta^{2} \cdot \text{precision}) + \text{recall}}$$
Sehingga 
$$f_{1} = (1 + 1^{2}) \cdot \frac{\text{precition} \cdot \text{recall}}{(1^{2} \cdot \text{precision}) + \text{recall}}$$
$$f_{1} = (2) \cdot \frac{\text{precition} \cdot \text{recall}}{(\text{precision}) + \text{recall}}$$



In [66]:
def f1_score(y_true, y_pred):
  precision_val = precision(y_true, y_pred)
  recall_val = recall(y_true, y_pred)
  return 2*((precision_val * recall_val)/(precision_val + recall_val + K.epsilon()))

**n.b.: Semua fungsi pada bagian penyebut ditambah epsilon untuk menghindari dibagi $0$ dikarenakan memasukkan input sebanyak $0$**

##### Fungsi Bantuan untuk _Load Dataset_

In [67]:
def load_dataset():
	(trainX, trainY), (testX, testY) = keras.datasets.cifar10.load_data()
	trainY = tf.keras.utils.to_categorical(trainY) ## Di ubah ke matrix persegi representasi binari
	testY = tf.keras.utils.to_categorical(testY) ## Di ubah ke matrix persegi representasi binari
	return trainX, trainY, testX, testY

#### Load Dataset

In [17]:
x_train, y_train, x_test, y_test = load_dataset()

##### Membangun Model _Fully Connected Neural Network_ dengan spesifikasi minimum memenuhi deskripsi Lab

In [70]:
## Melakukan inisialisasi node per layer
nodes_per_layer = [1000, 1000, 10]
activation_function_per_layer = [tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
layers = []
layers.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_layer)):
  node = nodes_per_layer[i]
  activation = activation_function_per_layer[i]
  layers.append(keras.layers.Dense(node, activation=activation))
  if i != len(nodes_per_layer)-1:
    ## Menambahkan dropout regularization di antara dua layer
    layers.append(keras.layers.Dropout(0.2))

## Mendefinisikan model
model = Sequential(layers)

## Membangun model
model.build(input_shape=x_train.shape)
model.output_shape

(50000, 10)

In [71]:
## Melakukan kompilasi ke dalam model
model.compile(
    optimizer=tf.optimizers.Adam(0.001),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

In [ ]:
## Melakukan fitting
beginning_hist = model.fit(x_train, y_train, epochs=5, batch_size=2000, validation_data=(x_test,y_test), use_multiprocessing=True)

Epoch 1/5
25/25 [==============================] - 18s 721ms/step - loss: 377.7602 - accuracy: 0.1150 - recall: 0.1050 - specificity: 0.9102 - f1_score: 0.1082 - val_loss: 2.3548 - val_accuracy: 0.1158 - val_recall: 0.0124 - val_specificity: 0.9952 - val_f1_score: 0.0235
Epoch 2/5
25/25 [==============================] - 18s 706ms/step - loss: 2.3234 - accuracy: 0.1017 - recall: 0.0021 - specificity: 0.9991 - f1_score: 0.0040 - val_loss: 2.3011 - val_accuracy: 0.1003 - val_recall: 1.0000e-04 - val_specificity: 1.0000 - val_f1_score: 1.9990e-04
Epoch 3/5
25/25 [==============================] - 18s 704ms/step - loss: 2.3066 - accuracy: 0.1005 - recall: 2.8000e-04 - specificity: 0.9999 - f1_score: 5.5858e-04 - val_loss: 2.3013 - val_accuracy: 0.1007 - val_recall: 1.0000e-04 - val_specificity: 1.0000 - val_f1_score: 1.9990e-04
Epoch 4/5
19/25 [=====================>........] - ETA: 3s - loss: 2.3037 - accuracy: 0.1003 - recall: 2.6316e-04 - specificity: 0.9999 - f1_score: 5.2555e-04

In [26]:
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.regularizers import l2

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.build(input_shape=x_train.shape)
print(model.output_shape)
# tmp_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

(None, 10)
